# [GA360] Conversion paths

## Initiate connection

In [1]:
# Import libraries
import pandas as pd
from google.cloud import bigquery

# Launch BigQuery client from the JSON credentials file
bq_client = bigquery.Client.from_service_account_json('../bigquery_creds.json')

# Helper function for queries to be returned as a Dataframe
def bquery(sql):
    return bq_client.query(sql).to_dataframe()

## Path length: how many sessions before converting

In [2]:
# Get number of session before a transaction, and conversion rates by session position
sql = """
WITH sub1 AS (
    SELECT 
        fullVisitorId,
        CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS session_id,
        visitStartTime,
        MAX(IF(totals.transactions > 0, 1, 0)) AS transaction_session,
        RANK() OVER (PARTITION BY fullVisitorId ORDER BY visitStartTime) AS session_number
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE 
        _TABLE_SUFFIX BETWEEN '20170701' AND '20170710'
    GROUP BY fullVisitorId, session_id, visitStartTime
)

SELECT
    session_number,
    COUNT(session_id) AS sessions,
    SUM(transaction_session) AS transactions,
    SUM(transaction_session)/COUNT(session_id) AS conversion_rate
FROM sub1
GROUP BY session_number
HAVING transactions > 0
ORDER BY session_number
"""
bquery(sql)

,session_number,sessions,transactions,conversion_rate
0,1,19671,141,0.007168
1,2,1859,43,0.023131
2,3,520,17,0.032692
3,4,195,7,0.035897
4,5,99,6,0.060606
5,6,41,2,0.048780
6,10,10,1,0.100000


## Time lag: how much time before converting

In [3]:
# Get time lag (in days) between first session of a user and a converting session
sql = """
WITH sub1 AS (
    SELECT 
        fullVisitorId,
        MIN(visitStartTime) OVER (PARTITION BY fullVisitorId ORDER BY visitStartTime) AS first_session_time,
        MAX(IF(totals.transactions > 0, visitStartTime, 0)) AS transaction_session_time
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE 
        _TABLE_SUFFIX BETWEEN '20170701' AND '20170710'
    GROUP BY fullVisitorId, visitStartTime
)

SELECT
    FLOOR((transaction_session_time - first_session_time)/3600/24) AS lag_days,
    COUNT(*) AS transactions
FROM sub1
WHERE transaction_session_time > 0
GROUP BY lag_days
ORDER BY lag_days
"""
bquery(sql)

,lag_days,transactions
0,0.0,187
1,1.0,12
2,2.0,5
3,3.0,4
4,4.0,5
5,5.0,2
6,6.0,1
7,7.0,1


## Differentiate brand vs non-brand keywords

In [4]:
# Differentiate between SEA keywords containing 'google' vs others
sql = """
SELECT 
    CASE
        WHEN REGEXP_CONTAINS(LOWER(trafficSource.keyword), r"google") AND channelGrouping = 'Paid Search' THEN 'Paid Search Brand'
        WHEN channelGrouping = 'Paid Search' THEN 'Paid Search Non Brand'
        ELSE 'Other channels'
    END AS channel,
    SUM(totals.visits) AS sessions,
    SUM(totals.transactions) AS transactions,
    SUM(totals.transactions)/SUM(totals.visits) AS conversion_rate,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE 
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170710'
GROUP BY 1
ORDER BY 1
"""
bquery(sql)

,channel,sessions,transactions,conversion_rate
0,Other channels,21698,217,0.010001
1,Paid Search Brand,376,1,0.002660
2,Paid Search Non Brand,395,8,0.020253


## True Direct vs Last Non Direct Click

Definition of "Direct" channel in Google Analytics is rather narrow. A user will only be considered coming through "Direct" if he/she did *not* previously came via *any* other channel. Hence, acquisition channels in GA are attributed to the Last Non-Direct click. 

With the BigQuery raw data, you can know if a user *actually* came directly to your site, even if he/she previously arrived via another channel, with the `trafficSource.isTrueDirect` field. Note how conversion rates are much higher when considering this parameter.

In [5]:
# Differentiate True Direct in channels
sql = """
SELECT 
    channelGrouping AS channel,
    trafficSource.isTrueDirect IS NOT NULL AS true_direct,
    SUM(totals.visits) AS sessions,
    SUM(totals.transactions) AS transactions,
    ROUND(100*SUM(totals.transactions)/SUM(totals.visits), 2) AS conversion_rate,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE 
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY 1,2
ORDER BY 1,2
"""
bquery(sql)

,channel,true_direct,sessions,transactions,conversion_rate
0,(Other),False,1,NaN,NaN
1,Affiliates,False,1460,2.0,0.14
2,Affiliates,True,328,2.0,0.61
3,Direct,True,12306,131.0,1.06
4,Display,False,548,14.0,2.55
5,Display,True,142,7.0,4.93
6,Organic Search,False,31468,168.0,0.53
7,Organic Search,True,6187,145.0,2.34
8,Paid Search,False,1684,37.0,2.20
9,Paid Search,True,421,16.0,3.80
